# 🤖 Entrenamiento de Modelos ML - Econova
## Plataforma Inteligente de Simulación Financiera

**Autor:** Diego (Responsable de Machine Learning)  
**Proyecto:** Econova - Simulación Financiera Empresarial con IA

### Objetivos:
1. Entrenar modelos de predicción de **ventas/ingresos**
2. Entrenar modelos de predicción de **crecimiento empresarial**
3. Entrenar modelos de **clasificación de riesgo financiero**
4. Exportar modelos para uso via API

### Tecnologías:
- **scikit-learn**: Modelos base (Random Forest, Gradient Boosting)
- **XGBoost**: Modelo avanzado para mejor rendimiento
- **pandas/NumPy**: Procesamiento de datos

## 1. Importación de Librerías

In [ ]:
# Librerías principales
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)
import xgboost as xgb

# Serialización de modelos
import joblib
import os
from datetime import datetime

# Configuración
np.random.seed(42)
print("✅ Librerías importadas correctamente")
print(f"📅 Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Generación de Datos Sintéticos para Entrenamiento

Como es un proyecto de demostración, generaremos datos sintéticos realistas que simulan:
- **Datos financieros de empresas**: ingresos, gastos, activos, pasivos
- **Indicadores de rendimiento**: ROI, margen de ganancia, liquidez
- **Variables macroeconómicas**: inflación, tasa de interés

In [ ]:
def generar_datos_financieros(n_empresas=1000):
    """
    Genera datos sintéticos de empresas para entrenamiento de modelos ML.
    
    Returns:
        DataFrame con datos financieros de empresas simuladas
    """
    np.random.seed(42)
    
    # Sectores empresariales
    sectores = ['Tecnología', 'Manufactura', 'Servicios', 'Comercio', 'Agricultura']
    tamanios = ['Micro', 'Pequeña', 'Mediana', 'Grande']
    
    data = {
        # Identificación
        'sector': np.random.choice(sectores, n_empresas),
        'tamanio': np.random.choice(tamanios, n_empresas, p=[0.4, 0.3, 0.2, 0.1]),
        'antiguedad_anios': np.random.randint(1, 30, n_empresas),
        
        # Financieros básicos (en miles de soles)
        'ingresos_anuales': np.random.exponential(500, n_empresas) + 50,
        'gastos_operativos': None,  # Se calculará
        'activos_totales': np.random.exponential(800, n_empresas) + 100,
        'pasivos_totales': None,  # Se calculará
        'patrimonio': None,  # Se calculará
        
        # Indicadores de mercado
        'num_empleados': np.random.randint(1, 500, n_empresas),
        'num_clientes': np.random.randint(10, 10000, n_empresas),
        'tasa_retencion_clientes': np.random.uniform(0.5, 0.98, n_empresas),
        
        # Variables macroeconómicas (simuladas)
        'inflacion': np.random.uniform(0.02, 0.08, n_empresas),
        'tasa_interes_referencia': np.random.uniform(0.03, 0.12, n_empresas),
        'crecimiento_pib_sector': np.random.uniform(-0.02, 0.08, n_empresas),
    }
    
    df = pd.DataFrame(data)
    
    # Calcular variables dependientes con lógica financiera realista
    df['gastos_operativos'] = df['ingresos_anuales'] * np.random.uniform(0.6, 0.95, n_empresas)
    df['pasivos_totales'] = df['activos_totales'] * np.random.uniform(0.3, 0.8, n_empresas)
    df['patrimonio'] = df['activos_totales'] - df['pasivos_totales']
    
    # Indicadores calculados
    df['margen_bruto'] = (df['ingresos_anuales'] - df['gastos_operativos']) / df['ingresos_anuales']
    df['ratio_endeudamiento'] = df['pasivos_totales'] / df['activos_totales']
    df['roi'] = (df['ingresos_anuales'] - df['gastos_operativos']) / df['activos_totales']
    df['liquidez'] = df['activos_totales'] / (df['pasivos_totales'] + 1)
    
    # Variable objetivo: Crecimiento esperado (influenciado por múltiples factores)
    df['crecimiento_anual'] = (
        0.02 +  # Base
        df['margen_bruto'] * 0.15 +
        df['roi'] * 0.10 +
        df['crecimiento_pib_sector'] * 0.5 +
        df['tasa_retencion_clientes'] * 0.08 -
        df['ratio_endeudamiento'] * 0.05 -
        df['inflacion'] * 0.3 +
        np.random.normal(0, 0.03, n_empresas)  # Ruido
    )
    
    # Clasificación de riesgo (basada en indicadores)
    condiciones_riesgo = [
        (df['ratio_endeudamiento'] > 0.7) | (df['margen_bruto'] < 0.1) | (df['liquidez'] < 1.2),
        (df['ratio_endeudamiento'] > 0.5) | (df['margen_bruto'] < 0.2),
        (df['ratio_endeudamiento'] <= 0.5) & (df['margen_bruto'] >= 0.2)
    ]
    categorias_riesgo = ['Alto', 'Medio', 'Bajo']
    df['nivel_riesgo'] = np.select(condiciones_riesgo, categorias_riesgo, default='Medio')
    
    # Predicción de ingresos próximo año
    df['ingresos_proximo_anio'] = df['ingresos_anuales'] * (1 + df['crecimiento_anual'])
    
    return df

# Generar datos
df_empresas = generar_datos_financieros(1500)
print(f"✅ Dataset generado: {df_empresas.shape[0]} empresas, {df_empresas.shape[1]} variables")
print(f"\n📊 Distribución de riesgo:")
print(df_empresas['nivel_riesgo'].value_counts())
df_empresas.head()

In [ ]:
# Análisis exploratorio rápido
print("📈 Estadísticas descriptivas de variables numéricas clave:")
print(df_empresas[['ingresos_anuales', 'margen_bruto', 'roi', 'crecimiento_anual', 'ratio_endeudamiento']].describe().round(3))

## 3. Modelo 1: Predicción de Ingresos (Regresión)

Entrenaremos un modelo para predecir los **ingresos del próximo año** basado en indicadores financieros actuales.

### Características (Features):
- Ingresos actuales, gastos, activos, pasivos
- Indicadores: ROI, margen bruto, liquidez, ratio de endeudamiento
- Variables externas: inflación, tasa de interés, crecimiento del sector

In [ ]:
# Preparación de datos para predicción de ingresos
features_ingresos = [
    'ingresos_anuales', 'gastos_operativos', 'activos_totales', 'pasivos_totales',
    'antiguedad_anios', 'num_empleados', 'num_clientes', 'tasa_retencion_clientes',
    'margen_bruto', 'ratio_endeudamiento', 'roi', 'liquidez',
    'inflacion', 'tasa_interes_referencia', 'crecimiento_pib_sector'
]

X_ingresos = df_empresas[features_ingresos].copy()
y_ingresos = df_empresas['ingresos_proximo_anio'].copy()

# División train/test
X_train_ing, X_test_ing, y_train_ing, y_test_ing = train_test_split(
    X_ingresos, y_ingresos, test_size=0.2, random_state=42
)

# Escalado de características
scaler_ingresos = StandardScaler()
X_train_ing_scaled = scaler_ingresos.fit_transform(X_train_ing)
X_test_ing_scaled = scaler_ingresos.transform(X_test_ing)

print(f"✅ Datos preparados:")
print(f"   - Train: {X_train_ing.shape[0]} muestras")
print(f"   - Test: {X_test_ing.shape[0]} muestras")
print(f"   - Features: {len(features_ingresos)}")

In [ ]:
# Entrenar y comparar múltiples modelos de regresión
modelos_regresion = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42, verbosity=0)
}

resultados_regresion = []

print("🔄 Entrenando modelos de regresión para predicción de ingresos...\n")

for nombre, modelo in modelos_regresion.items():
    # Entrenar
    if nombre == 'Linear Regression':
        modelo.fit(X_train_ing_scaled, y_train_ing)
        y_pred = modelo.predict(X_test_ing_scaled)
    else:
        modelo.fit(X_train_ing, y_train_ing)
        y_pred = modelo.predict(X_test_ing)
    
    # Métricas
    rmse = np.sqrt(mean_squared_error(y_test_ing, y_pred))
    mae = mean_absolute_error(y_test_ing, y_pred)
    r2 = r2_score(y_test_ing, y_pred)
    
    resultados_regresion.append({
        'Modelo': nombre,
        'RMSE': round(rmse, 2),
        'MAE': round(mae, 2),
        'R²': round(r2, 4)
    })
    
    print(f"📊 {nombre}:")
    print(f"   RMSE: {rmse:.2f} | MAE: {mae:.2f} | R²: {r2:.4f}")
    print()

# Tabla comparativa
df_resultados_reg = pd.DataFrame(resultados_regresion)
print("\n📈 Comparación de Modelos de Regresión:")
print(df_resultados_reg.to_string(index=False))

In [ ]:
# Seleccionar el mejor modelo (XGBoost generalmente)
mejor_modelo_ingresos = modelos_regresion['XGBoost']

# Importancia de características
importancia_ingresos = pd.DataFrame({
    'Feature': features_ingresos,
    'Importancia': mejor_modelo_ingresos.feature_importances_
}).sort_values('Importancia', ascending=False)

print("🎯 Importancia de Variables para Predicción de Ingresos:")
print(importancia_ingresos.to_string(index=False))

## 4. Modelo 2: Predicción de Crecimiento Empresarial (Regresión)

Modelo para predecir la **tasa de crecimiento anual** de una empresa basada en sus indicadores financieros y de mercado.

In [ ]:
# Preparación de datos para predicción de crecimiento
features_crecimiento = [
    'ingresos_anuales', 'gastos_operativos', 'activos_totales', 
    'antiguedad_anios', 'num_empleados', 'tasa_retencion_clientes',
    'margen_bruto', 'ratio_endeudamiento', 'roi', 'liquidez',
    'inflacion', 'tasa_interes_referencia', 'crecimiento_pib_sector'
]

X_crec = df_empresas[features_crecimiento].copy()
y_crec = df_empresas['crecimiento_anual'].copy()

# División train/test
X_train_crec, X_test_crec, y_train_crec, y_test_crec = train_test_split(
    X_crec, y_crec, test_size=0.2, random_state=42
)

# Entrenar modelo XGBoost para crecimiento
modelo_crecimiento = xgb.XGBRegressor(
    n_estimators=150,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    verbosity=0
)
modelo_crecimiento.fit(X_train_crec, y_train_crec)

# Evaluar
y_pred_crec = modelo_crecimiento.predict(X_test_crec)
rmse_crec = np.sqrt(mean_squared_error(y_test_crec, y_pred_crec))
mae_crec = mean_absolute_error(y_test_crec, y_pred_crec)
r2_crec = r2_score(y_test_crec, y_pred_crec)

print("📊 Modelo de Predicción de Crecimiento (XGBoost):")
print(f"   RMSE: {rmse_crec:.4f}")
print(f"   MAE: {mae_crec:.4f}")
print(f"   R²: {r2_crec:.4f}")

# Importancia de características
importancia_crec = pd.DataFrame({
    'Feature': features_crecimiento,
    'Importancia': modelo_crecimiento.feature_importances_
}).sort_values('Importancia', ascending=False)

print("\n🎯 Importancia de Variables para Predicción de Crecimiento:")
print(importancia_crec.head(10).to_string(index=False))

## 5. Modelo 3: Clasificación de Riesgo Financiero

Modelo de **clasificación** para determinar el nivel de riesgo de una empresa:
- **Bajo**: Empresa financieramente saludable
- **Medio**: Riesgo moderado, requiere atención
- **Alto**: Riesgo elevado, necesita intervención urgente

In [ ]:
# Preparación de datos para clasificación de riesgo
features_riesgo = [
    'ingresos_anuales', 'gastos_operativos', 'activos_totales', 'pasivos_totales',
    'antiguedad_anios', 'num_empleados', 'tasa_retencion_clientes',
    'margen_bruto', 'ratio_endeudamiento', 'roi', 'liquidez',
    'inflacion', 'tasa_interes_referencia'
]

X_riesgo = df_empresas[features_riesgo].copy()
y_riesgo = df_empresas['nivel_riesgo'].copy()

# Codificar etiquetas
label_encoder_riesgo = LabelEncoder()
y_riesgo_encoded = label_encoder_riesgo.fit_transform(y_riesgo)

# División train/test
X_train_riesgo, X_test_riesgo, y_train_riesgo, y_test_riesgo = train_test_split(
    X_riesgo, y_riesgo_encoded, test_size=0.2, random_state=42, stratify=y_riesgo_encoded
)

print(f"✅ Datos preparados para clasificación de riesgo")
print(f"   Clases: {label_encoder_riesgo.classes_}")
print(f"   Train: {len(y_train_riesgo)} | Test: {len(y_test_riesgo)}")

In [ ]:
# Entrenar modelos de clasificación
modelos_clasificacion = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'XGBoost': xgb.XGBClassifier(n_estimators=100, random_state=42, verbosity=0, use_label_encoder=False)
}

resultados_clasificacion = []

print("🔄 Entrenando modelos de clasificación de riesgo...\n")

for nombre, modelo in modelos_clasificacion.items():
    # Entrenar
    modelo.fit(X_train_riesgo, y_train_riesgo)
    y_pred = modelo.predict(X_test_riesgo)
    
    # Métricas
    acc = accuracy_score(y_test_riesgo, y_pred)
    prec = precision_score(y_test_riesgo, y_pred, average='weighted')
    rec = recall_score(y_test_riesgo, y_pred, average='weighted')
    f1 = f1_score(y_test_riesgo, y_pred, average='weighted')
    
    resultados_clasificacion.append({
        'Modelo': nombre,
        'Accuracy': round(acc, 4),
        'Precision': round(prec, 4),
        'Recall': round(rec, 4),
        'F1-Score': round(f1, 4)
    })
    
    print(f"📊 {nombre}:")
    print(f"   Accuracy: {acc:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f} | F1: {f1:.4f}")
    print()

# Tabla comparativa
df_resultados_clf = pd.DataFrame(resultados_clasificacion)
print("\n📈 Comparación de Modelos de Clasificación:")
print(df_resultados_clf.to_string(index=False))

In [ ]:
# Mejor modelo de clasificación
mejor_modelo_riesgo = modelos_clasificacion['XGBoost']
y_pred_riesgo = mejor_modelo_riesgo.predict(X_test_riesgo)

# Reporte de clasificación detallado
print("📋 Reporte de Clasificación Detallado (XGBoost):\n")
print(classification_report(
    y_test_riesgo, 
    y_pred_riesgo, 
    target_names=label_encoder_riesgo.classes_
))

# Matriz de confusión
print("\n🔢 Matriz de Confusión:")
cm = confusion_matrix(y_test_riesgo, y_pred_riesgo)
cm_df = pd.DataFrame(cm, 
                     index=label_encoder_riesgo.classes_, 
                     columns=label_encoder_riesgo.classes_)
print(cm_df)

## 6. Exportación de Modelos para API

Guardaremos los modelos entrenados para su uso en la API del backend.

In [ ]:
# Crear directorio para modelos si no existe
modelos_dir = os.path.join(os.path.dirname(os.getcwd()), 'ml', 'modelos')
if not os.path.exists(modelos_dir):
    os.makedirs(modelos_dir)
    print(f"📁 Directorio creado: {modelos_dir}")

# Guardar modelos
modelos_a_guardar = {
    'modelo_ingresos_xgb.joblib': mejor_modelo_ingresos,
    'modelo_crecimiento_xgb.joblib': modelo_crecimiento,
    'modelo_riesgo_xgb.joblib': mejor_modelo_riesgo,
    'scaler_ingresos.joblib': scaler_ingresos,
    'label_encoder_riesgo.joblib': label_encoder_riesgo
}

for nombre_archivo, modelo in modelos_a_guardar.items():
    ruta_completa = os.path.join(modelos_dir, nombre_archivo)
    joblib.dump(modelo, ruta_completa)
    print(f"✅ Guardado: {nombre_archivo}")

# Guardar metadatos de features
metadatos = {
    'features_ingresos': features_ingresos,
    'features_crecimiento': features_crecimiento,
    'features_riesgo': features_riesgo,
    'clases_riesgo': list(label_encoder_riesgo.classes_),
    'fecha_entrenamiento': datetime.now().isoformat(),
    'version': '1.0.0'
}

joblib.dump(metadatos, os.path.join(modelos_dir, 'metadatos_modelos.joblib'))
print(f"✅ Guardado: metadatos_modelos.joblib")

print(f"\n🎉 Todos los modelos exportados exitosamente en: {modelos_dir}")

## 7. Resumen y Conclusiones

### Modelos Entrenados:
1. **Predicción de Ingresos** (XGBoost Regressor) - Predice ingresos del próximo año
2. **Predicción de Crecimiento** (XGBoost Regressor) - Predice tasa de crecimiento anual
3. **Clasificación de Riesgo** (XGBoost Classifier) - Clasifica en Bajo/Medio/Alto riesgo

### Archivos Exportados:
- `modelo_ingresos_xgb.joblib` - Modelo de predicción de ingresos
- `modelo_crecimiento_xgb.joblib` - Modelo de predicción de crecimiento
- `modelo_riesgo_xgb.joblib` - Modelo de clasificación de riesgo
- `scaler_ingresos.joblib` - Escalador para features de ingresos
- `label_encoder_riesgo.joblib` - Codificador de etiquetas de riesgo
- `metadatos_modelos.joblib` - Información de features y configuración

### Próximos Pasos:
- Integrar modelos con el backend via `ml_servicio.py`
- Implementar endpoints en `rutas/ml.py`
- Conectar con el análisis de sensibilidad